In [1]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Hansen
[nltk_data]     Dafa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Hansen
[nltk_data]     Dafa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Data dokumen
docs = {
    'D1': "Machine learning improves search engines.",
    'D2': "Information retrieval techniques are evolving.",
    'D3': "Search engines use advanced algorithms.",
    'D4': "Deep learning and neural networks are popular.",
    'D5': "Boolean retrieval uses logical operators.",
    'D6': "Query processing is essential in search engines.",
    'D7': "Text mining and NLP are related to information retrieval.",
    'D8': "Search algorithms improve information discovery.",
    'D9': "Data science leverages machine learning.",
    'D10': "Ranking methods optimize search engine results."
}


In [ ]:
# Daftar query
queries = [
    ("Search AND Engine", "AND"),
    ("Information OR Retrieval", "OR"),
    ("Machine NOT Learning", "NOT")
]